# Решение кейса CUP IT от команды "Аня 24"

## Загрузка требуемых библиотек и данных( с первичными преобразованиями)

In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sea
from datetime import datetime
%matplotlib inline 

In [4]:
#считываем данные, объединяем две таблицы по Id покупателя, удаляем ненужную колонку "enr_date,также переводим дату в нужный тип 
data_customers=pd.read_csv('F:\Downloads\Cup_IT_Датасет_Аналитика_данных_cross-sell_model\data\png_magnit_customers.csv')
data_purchases=pd.read_csv('F:\Downloads\Cup_IT_Датасет_Аналитика_данных_cross-sell_model\data\png_magnit_purchases.csv')
data = data_purchases.merge(data_customers).drop(['enr_date'],axis=1).rename(columns={'purchase date':'purchase_date'})
data['purchase_date']=pd.to_datetime(data['purchase_date'])

In [5]:
# добавляем столбец с номером месяца, чтобы затем строить временные ряды
data['purchase_mounth'] = [data['purchase_date'].iloc[i].month for i in range(data.shape[0])] 

In [7]:
#Удаляем все строки, где продукт - еда, так как они не учавствуют в нашем анализе, по условию кейса
food=['Вафли','Йогурты питьевые','Круаcсаны','Печенье (фас)','Кукурузные палочки','Пшено','Воды минеральные','Шоколадные конфеты в коробках',\
     'Пряничные изделия','Снэки из рыбы и морепродуктов','Газированные напитки российских брендов','Орехи (фас)',\
     'Попкорн','Семечки подсолнечника','Сухари-гренки','Чипсы','Сокосодержащие напитки','Мороженое порционное',\
     'Смеси сухие детские','Мясосодержащие консервы детские','Каши детские','Печенье, снэки детские','Йогурты детские',\
     'Энергетические напитки','Гематоген','Газированные питьевые воды','Готовое спортивное питание','Холодные чаи','Карты лояльности Магнит',\
     'Напитки детские','Негазированные питьевые воды','Газированные напитки импортных брендов','Пасты кондитерские',\
     'Шоколад плиточный','Сладкий снэк', 'Вкусовые питьевые воды',]
data = data[~data['product'].isin(food)]


## Построение сводных таблиц для построения графиков анализа
Графики затем строятся в Excel, так как там это делать проще и можно будет удобно корректировать их в презентации

In [306]:
#строим диаграмму с группировкой по полу и возрасту с показателем среднему позиций в чеке. 
diagramma_1 = data.groupby(['gender','customer_age_group','customer','purchase_date'], as_index = False).aggregate({'product':'count'})
diagramma_1 = diagramma_1.groupby(['gender','customer_age_group']).aggregate({'product':'mean'})
diagramma_1.to_csv('diagramma_1.csv')

In [308]:
#строим диаграмму с группировкой по полу с показателем количества покупок товаров P&G. 
diagramma_2 = data[data['p&g_product']>0].groupby('gender').aggregate({'customer':'count'})
diagramma_2.to_csv('diagramma_2.csv')

In [10]:
#строим диаграмму по временни с выделение подгруппы товаров P&G с показателем количества покупок товаров.
diagramma_3_all = data.groupby('purchase_mounth', as_index = False).aggregate({'cost_product':'count'})
diagramma_3_PG = data[data['p&g_product']>0].groupby('purchase_mounth', as_index = False).aggregate({'customer':'count'})
diagramma_3_all = diagramma_3_all.merge(diagramma_3_PG, on = 'purchase_mounth')
diagramma_3_all.to_csv('diagramma_3.csv')

In [44]:
#строим точечную диаграмму для всех продуктов, где по оси х - среднее кол-во позиций в чеке, где он учавствует
#по оси у - сколько раз товар встречался во всех чеках
diagramma_4 = data.groupby(['customer','purchase_date'],as_index= False).aggregate({'cost_product':'count'}).rename(columns={'cost_product':'dop_count'})
diagramma_4 = data.merge(diagramma_4, on = ['customer','purchase_date'])

diagramma_4_without_PG_x = diagramma_4[diagramma_4['p&g_product']==0].groupby(['product'],as_index= False).aggregate({'dop_count':'mean'})
diagramma_4_PG_x = diagramma_4[diagramma_4['p&g_product']>0].groupby(['product'],as_index= False).aggregate({'dop_count':'mean'})

diagramma_4_without_PG_y = data[data['p&g_product']==0].groupby('product',as_index= False).aggregate({'cost_product':'count'})
diagramma_4_PG_y = data[data['p&g_product']>0].groupby('product',as_index= False).aggregate({'cost_product':'count'})


diagramma_4_without_PG = diagramma_4_without_PG_x.merge(diagramma_4_without_PG_y, on = 'product')
diagramma_4_PG = diagramma_4_PG_x.merge(diagramma_4_PG_y, on = 'product')

diagramma_4_PG.to_csv('diagramma_4_PG.csv')
diagramma_4_without_PG.to_csv('diagramma_4_without_PG.csv')


diagramma_4_PG.to_csv('diagramma_4_PG.csv')
diagramma_4_without_PG.to_csv('diagramma_4_without_PG.csv')

In [50]:
#Диаграмма по каналам попадания покупателя в систему
diagramma_5 = data[data['p&g_product']>0].groupby('enr_channel').aggregate({'customer':'count'})
diagramma_5.to_csv('diagramma_5.csv')

## Создаем вспомагительные таблицы для дальнейшего анализа

In [17]:
#Таблица с всеми товарами, также индексируем их по возрастанию кол-ва его покупки
data_product = data.groupby('product').aggregate({'other_product':'count'}).sort_values('other_product')
data_product['index']=[i for i in range(data_product.shape[0])]

In [18]:
#Таблица с всеми чеками - несколько строк из data попадают в чек, если они были куплены одним пользователем в один день
cheques = data.groupby(['customer','purchase_date'])\
    .aggregate({'cost_product':'sum','cost_p&g':'sum' })\
    .rename(columns={'cost_product':'total_product', 'cost_p&g':'total_p&g'})
cheques['total_cost']=cheques['total_product']+cheques['total_p&g']
cheques = cheques.merge(data[['customer','purchase_date','gender','customer_age_group']], on=['customer','purchase_date'])

In [19]:
#Группировка чеков по разным группам - пол и возраст с показателем - средняя цена товаров (по z шкале)
cheques_by = cheques.groupby(['gender','customer_age_group']).aggregate({'total_cost':'mean'}).sort_values(['gender','total_cost'], ascending = False)

In [20]:
# Добавление индексов товаров в основную таблицу, создание таблицы чеков, но параметром as_index=False
data = data.merge(data_product, on='product')
cheques_position = data.groupby(['customer','purchase_date', 'product','index'],as_index=False).aggregate({'gender':'count'})b

In [27]:
#Таблица товаров P&G, к которым добавлена основная идексация товаров- см. выше 
PG_product = data[data['p&g_product']>0].groupby('product').aggregate({'customer':'count'})
PG_product = PG_product.merge(data_product, on = 'product')
PG_product = PG_product.drop(columns=['customer','other_product'])
PG_product = PG_product.reset_index(0)
mas_name_PG = [PG_product['product'][i] for i in range(PG_product.shape[0])]

## Создание матрицы встречаемости двух товаров в одном чеке

In [51]:
#matrx - двухмерный массив, в котором ячейка[i][j] - это сколько раз товары по i и j индексу встречались вместе в одном чеке
#благодаря группировке - асимптотика алгоритма примерно О(N) 
matrix = np.array([[0 for i in range(358)] for i in range(data_product.shape[0])])
matrix

i=0
while i < cheques_position.shape[0]-1:
    customer_id = cheques_position.iloc[i]['customer']
    while cheques_position.iloc[i]['customer'] == customer_id:
        mas=[]
        purchase = cheques_position.iloc[i]['purchase_date']
        while cheques_position.iloc[i]['purchase_date'] == purchase:
            mas.append(cheques_position.iloc[i]['index'])
            i+=1
        for j in mas:
            for k in mas:
                matrix[j][k]+=1

KeyboardInterrupt: 

In [ ]:
#преобразовываем матрицу в DataFrame, добавляем имена товаров и убираем товары,которые встречались меньше 1000 раз всего. 
#Их анализ будет нерелевантным. Но этого делать необязательно, просто итоговая таблица будет больше. Тут появляется ошибка-
#Но это просто предупреждение от Pandas, что можно сделать по-другому, всё работает
data_matrix= pd.DataFrame(matrix)
data_matrix= data_matrix.reset_index(0)

data_matrix= data_matrix.merge(data_product, on='index')
data_matrix= data_matrix.set_index('product')
data_matrix= data_matrix.drop(columns=['index','level_0','other_product_x'])
data_matrix= data_matrix.transpose()
data_matrix= data_matrix.reset_index(0)
for i in range(data_matrix.shape[0]):
     data_matrix=['index'][i]=data_product['product'][i]
data_matrix=  data_matrix.set_index('index')
data_matrix= data_matrix.drop(columns=['Одежда для купания','Туристическое снаряжение','Ирригаторы (включая расходные к ним материалы)','Бандажи дородовые и послеродовые','Спец.средства(уход за брекетами и протезами)',
 'Зимний спортивный инвентарь','Колготки','Педикулезные средства','Подарочные наборы','Автоэлектроника','Устройства для курения','Бараночные изделия','Спортивные игры','Бады для женского здоровья','Спринцовки и кружки Эсмарха','Товары для прогулки и отдыха','Журналы','Глюкометры (включая расходные к ним материалы)','Банки для консервирования','Средства для носовой полости','IT девайсы','Крупная бытовая техника',
 'Садовый интерьер','Велоспорт','Украшения для выпечки','Складная и пластиковая мебель','Рождественские елки','Бады для почек, печени','Пасхальная посуда','Чистящие средства для техники',
 'Аксессуары для кемпинга','Ингаляторы','Стремянки, лестницы','Бады для зрения','Повседневная обувь','Тонометры (включая расходные к ним материалы)','Косметические наборы','Бады для похудения','Бады для ЖКТ','Аксессуары для сада','Хлопушки',
 'Средства для лечения верхних дыхательных путей','Спортивный инвентарь Промо','Художественная литература','Аксессуарика','Мебель для террасы','Посадочный инвентарь',
 'Витамины таблетированные','Линейки сервировочной посуды фаянс','Бады для сердца, сосудов','Автозапчасти','Рюкзаки, сумки, чемоданы Промо','Витамины жидкие, растворимые',
 'Товары для безопасности','Аккумуляторы, зарядные устройства','Освещение','Тесты для диагностики заболеваний','Бады для красоты','Формы для куличей','Посадочный материал','Коллекции',
 'Инвентарь для полива','Тесты для определения овуляции','Строительный инвентарь','Технические жидкости','Сигареты','Климатическая техника','Искусственные растения',
 'Бады для иммунитета', 'Леденцы, пастилки','Папиросы','Средства ухода и защиты растений','Рабочая обувь','Посуда Промо','Инструменты для ухода за растениями','Живой срез','Электромонтаж',
 'Всесезонный спортивный инвентарь','Средства для розжига','Бассейны','Оборудование и инструменты','Напитки СП ММ','Защитные приспособления','Устройства ввода','Аксессуары для комнатных растений',
 'Подарочные книги','Творог детский','Бады для нервной системы','Защитные аксессуары','Аксессуары для купания','Электроинструменты и аксессуары к ним','Веники','Музыкальные инструменты','Цифровые носители информации','Приспособления для консервирования','Шарфы, палантины',
 'Надувные кровати','Функциональные напитки детские','Фонари'])
data_matrix = data_matrix.transpose()
data_matrix= data_matrix.drop(columns=['Одежда для купания','Туристическое снаряжение','Ирригаторы (включая расходные к ним материалы)','Бандажи дородовые и послеродовые','Спец.средства(уход за брекетами и протезами)',
 'Зимний спортивный инвентарь','Колготки','Педикулезные средства','Подарочные наборы','Автоэлектроника','Устройства для курения','Бараночные изделия','Спортивные игры','Бады для женского здоровья','Спринцовки и кружки Эсмарха','Товары для прогулки и отдыха','Журналы','Глюкометры (включая расходные к ним материалы)','Банки для консервирования','Средства для носовой полости','IT девайсы','Крупная бытовая техника',
 'Садовый интерьер','Велоспорт','Украшения для выпечки','Складная и пластиковая мебель','Рождественские елки','Бады для почек, печени','Пасхальная посуда','Чистящие средства для техники',
 'Аксессуары для кемпинга','Ингаляторы','Стремянки, лестницы','Бады для зрения','Повседневная обувь','Тонометры (включая расходные к ним материалы)','Косметические наборы','Бады для похудения','Бады для ЖКТ','Аксессуары для сада','Хлопушки',
 'Средства для лечения верхних дыхательных путей','Спортивный инвентарь Промо','Художественная литература','Аксессуарика','Мебель для террасы','Посадочный инвентарь',
 'Витамины таблетированные','Линейки сервировочной посуды фаянс','Бады для сердца, сосудов','Автозапчасти','Рюкзаки, сумки, чемоданы Промо','Витамины жидкие, растворимые',
 'Товары для безопасности','Аккумуляторы, зарядные устройства','Освещение','Тесты для диагностики заболеваний','Бады для красоты','Формы для куличей','Посадочный материал','Коллекции',
 'Инвентарь для полива','Тесты для определения овуляции','Строительный инвентарь','Технические жидкости','Сигареты','Климатическая техника','Искусственные растения',
 'Бады для иммунитета', 'Леденцы, пастилки','Папиросы','Средства ухода и защиты растений','Рабочая обувь','Посуда Промо','Инструменты для ухода за растениями','Живой срез','Электромонтаж',
 'Всесезонный спортивный инвентарь','Средства для розжига','Бассейны','Оборудование и инструменты','Напитки СП ММ','Защитные приспособления','Устройства ввода','Аксессуары для комнатных растений',
 'Подарочные книги','Творог детский','Бады для нервной системы','Защитные аксессуары','Аксессуары для купания','Электроинструменты и аксессуары к ним','Веники','Музыкальные инструменты','Цифровые носители информации','Приспособления для консервирования','Шарфы, палантины',
 'Надувные кровати','Функциональные напитки детские','Фонари'])

In [ ]:
#Сверху мы оставляем только товары от P&G - ведь, мы рекомендуем только товары данной компании
#Также заменяем кол-во пересечений товаров в одном чеке на показатель процентный показатель, сколько раз товар 
#встречался именно с ним в чеке, относительно других
#Сохраняем таблицу в csv, чтобы затем можно было с ней работать дальше
data_matrix = data_matrix.reset_index(0)
data_matrix = data_matrix[data_matrix['product'].isin(mas_name_PG)]
data_matrix = data_matrix.transpose()
data_matrix = data_matrix.reset_index(0)
data_matrix = data_matrix[~data_matrix['index'].isin(mas_name_PG)]
data_matrix = data_matrix.set_index('index')
sum_str = list(data_matrix.sum(axis=1))

for i in range(1,data_matrix.shape[0]):
    max_str = sum_str[i]
    for j in range(data_matrix.shape[1]):
        data_matrix.iat[i,j]=round((data_matrix.iat[i,j]/max_str)*100,2)
data_matrix.to_csv('final.csv')